In [1]:
import pandas as pd
import seaborn as sns
import os
import sys
import re
import concurrent.futures

from matplotlib import pyplot as plt
from dotenv import load_dotenv
from collections import defaultdict
from dataclasses import dataclass, asdict

sys.path.append("../")

from models.work_met import Work_met

load_dotenv()
sns.color_palette('colorblind')
plt.style.use('Solarize_Light2')

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not

try:
    pc_dpi = int(os.getenv('DPI'))
except TypeError:
    pc_dpi = 100
if pc_dpi is None:
    pc_dpi = 100


In [2]:
def get_images(dir: str, exclusion_list: list | None = None):
    """Extracts  the 'work names' of the meteorites based on the pictures loaded."""
    images = [file for file in os.listdir(dir) if str(file).endswith(".jpg")]
    if exclusion_list is not None:
        for excluded in exclusion_list:
            images.remove(excluded)

    return images


def get_work_names(image_list) -> dict:
    """Returns a dict with the expected 'work names' of the meteorites' images' names in image_list."""

    wname_fname_map = defaultdict(list)  # avoids checking for .keys() and initiates items as lists
    for image_name in image_list:
        # Remove file extension and parts after - or _
        work_name = image_name.split(".")[0].split("-")[0].split("_")[0].split("+")[0]
        
        # Clean up the work name if there are more than one alpha character after the last digit
        if not re.search(r'\d[a-zA-Z]{0,1}$', work_name):
            work_name = re.sub(r'(\d)[a-zA-Z]*$', r'\1', work_name)
        
        # Append the image name to the list associated with the work name
        wname_fname_map[work_name].append(image_name)

    return dict(wname_fname_map)



In [3]:
images_paths = get_images(dir="../imgs/")

print(f"{len(images_paths)} images detected")


704 images detected


In [4]:
work_names = get_work_names(image_list=images_paths)

print(f"{len(work_names)} meteorites detected")


230 meteorites detected


In [5]:
work_met_list = []
for name in work_names.keys():
    work_met_list.append(Work_met(work_name=name, images=work_names[name]))


In [6]:
def get_types_threads(work_met_list, n_threads = 25):
    """
    Uses multithreading to perform concurrent requests via the method Work_met.request_type()
    """
    if n_threads > 25:
        print(f"Thread limit of {n_threads} > 25, reducing it to 25 for fair use of the app")
        n_threads = 25

    for work_met in work_met_list:
        if not isinstance(work_met, Work_met):
            raise TypeError(f"At least one of the objects ({work_met}) is not of the class Meteorite")

    with concurrent.futures.ThreadPoolExecutor(max_workers=n_threads) as executor:
        futures = [executor.submit(work_met.request_type) for work_met in work_met_list]

        # Hopefully no error but this should be useful for debug
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"Error occurred: {e}")


In [7]:
get_types_threads(work_met_list=work_met_list, n_threads=25)


In [8]:
extract_failures = []

for work_met in work_met_list:
    try:
        if work_met.mtype is not None:
            pass
        else:
            extract_failures.append(work_met.work_name)
    except AttributeError:
        extract_failures.append(work_met.work_name)


In [9]:
print(extract_failures.__len__())


18


In [10]:
@dataclass
class Work_met_data:
    work_name: str
    mtype: str
    images: list

data_work_met = []

for work_met in work_met_list:
    data_work_met.append(Work_met_data(work_name=work_met.work_name, mtype=work_met.mtype, images=work_met.images))


In [11]:
df_work_met = pd.DataFrame(data=[asdict(work_met) for work_met in data_work_met])


## Identified unique types :

In [12]:
# Unique types :
display(df_work_met["mtype"].value_counts(dropna=False))


mtype
H5       84
L6       42
H6       34
LL5      21
None     18
H5-6     10
LL6       8
L5        7
LL5-6     1
L5-6      1
CV3       1
L4        1
L3.8      1
LL4       1
Name: count, dtype: int64

21 (len(extract_failures)) as none is to be expected. These have to be either fixed as request or manually. <br>
The rest of the types needs to be reduced into common classes or excluded to avoid lack of support in classification.

In [13]:
display(df_work_met[df_work_met["mtype"].isna()])


,work_name,mtype,images
57,GC029,None,"[GC029_T5-2.jpg, GC029_T5.jpg, GC029-T5.jpg]"
71,GC020,None,[GC020-TROILITE.jpg]
81,GC234,None,"[GC234-3.jpg, GC234-.jpg, GC234.jpg, GC234-chr..."
87,GC015,None,"[GC015-3.jpg, GC015-2.jpg]"
88,GC023,None,"[GC023-odd-CHONDRULES.jpg, GC023.jpg, GC023-2...."
90,GC021,None,"[GC021_T5.jpg, GC021_T5-2.jpg]"
108,GC067,None,"[GC067_T5-2.jpg, GC067_T5.jpg]"
124,GC014b,None,"[GC014b-T5-2.jpg, GC014b_Odd-PLAGIO-rim.jpg, G..."
125,G209,None,[G209.jpg]
133,GC019,None,"[GC019.jpg, GC019-T5.jpg]"


### Manual search :
- Missing from website : G015, GB39,  GC : [002, 004, 005, 006, 007, 010, 014 (/, b, c), 015, 019, 021, 024, 029 (/, T5), 067, ], MA294, MC176, RM0833

# Saving Dataset :

In [14]:
df_work_met.to_csv(path_or_buf="../data/work_met_img_type_1.csv", index=None)
df_work_met.to_pickle(path="../data/work_met_img_type_1.pkl")
